In [2]:
import os
import gensim
from gensim.models import LsiModel
from gensim import models
from gensim import corpora
from gensim.utils import lemmatize
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import remove_stopwords, stem_text
from gensim.parsing.preprocessing import strip_numeric, strip_short,strip_multiple_whitespaces,strip_non_alphanum,strip_punctuation,strip_tags,preprocess_string
import pandas as pd
from gensim import similarities
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
from pprint import pprint

In [3]:
#read the data
corpus_dir = 'https://raw.githubusercontent.com/Ramaseshanr/anlp/master/corpus/bbc-text.csv'
df_corpus = pd.read_csv(corpus_dir,names=['category', 'text'])
corpus = df_corpus['text'].values.tolist()
corpus = corpus[1:]
my_filter = [
    lambda x: x.lower(), strip_tags, strip_punctuation,
    strip_multiple_whitespaces, strip_numeric,
    remove_stopwords, strip_short, stem_text
]

In [5]:
def preprocessing(corpus):

    for document in corpus:
        doc = strip_numeric(document)
        doc = remove_stopwords(doc)
        doc = strip_short(doc,3)
        #doc = stem_text(doc)
        doc = strip_punctuation(doc)
        strip_tags(doc)
        yield gensim.utils.tokenize(doc, lower=True)

In [13]:
texts = preprocessing(corpus)
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, keep_n=25000)


In [14]:
doc_term_matrix = [dictionary.doc2bow(tokens) for tokens in preprocessing(corpus)]
tfidf = models.TfidfModel(doc_term_matrix)
corpus_tfidf = tfidf[doc_term_matrix]

In [15]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary)  # initialize an LSI transformation ##LSI-Latent Semantic Indexing
pprint(lsi.print_topics(num_topics=5, num_words=7))

[(0,
  '0.137*"labour" + 0.123*"election" + 0.118*"blair" + 0.110*"brown" + '
  '0.096*"government" + 0.092*"party" + 0.092*"people"'),
 (1,
  '-0.286*"labour" + -0.250*"election" + -0.239*"blair" + -0.218*"brown" + '
  '-0.178*"party" + -0.146*"tax" + -0.136*"chancellor"'),
 (2,
  '0.148*"england" + -0.146*"mobile" + -0.116*"growth" + -0.111*"bn" + '
  '0.102*"wales" + 0.100*"game" + 0.098*"win"'),
 (3,
  '0.313*"film" + 0.174*"best" + 0.151*"awards" + -0.149*"england" + '
  '0.125*"award" + 0.110*"music" + 0.109*"oscar"'),
 (4,
  '-0.228*"film" + 0.185*"mobile" + -0.148*"best" + -0.132*"bn" + '
  '-0.131*"dollar" + -0.125*"economy" + -0.118*"m"')]
